In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML

In [2]:
!ls /home/pierre/scVI/results/models_comparison/oscar_runs/

brain_small  hemato	       mixed_50_dataset  nb_dataset  retina
cortex	     mixed_25_dataset  mixed_75_dataset  pbmc	     zinb_dataset


In [22]:
data_path = '/home/pierre/scVI/results/models_comparison/oscar_runs'
dataset_name = 'mixed_25_dataset'
data_files = sorted(glob.glob(os.path.join(data_path, '{}/*.csv'.format(dataset_name))))
for data_file in data_files:
    print(data_file)
data_names = ['nb', 'zifa_full', 'zinb']

/home/pierre/scVI/results/models_comparison/oscar_runs/mixed_25_dataset/nb_mixed_25_dataset.csv
/home/pierre/scVI/results/models_comparison/oscar_runs/mixed_25_dataset/zifa_full_mixed_25_dataset.csv
/home/pierre/scVI/results/models_comparison/oscar_runs/mixed_25_dataset/zinb_mixed_25_dataset.csv


In [28]:
df[['t_dropout_phi_gen_gene', 't_dropout_phi_real_gene']].head()

,t_dropout_phi_gen_gene,t_dropout_phi_real_gene
0,[0.30892857 0.29107143 0.29535714 ... 0.002857...,[[0.27428571 0.26214286 0.24678571 ... 0.26178...
1,[0.30892857 0.29107143 0.29535714 ... 0.002857...,[[0.67392857 0.69785714 0.67392857 ... 0.6925 ...
2,[0.30892857 0.29107143 0.29535714 ... 0.002857...,[[0.59214286 0.57571429 0.5825 ... 0.58321...
3,[0.30892857 0.29107143 0.29535714 ... 0.002857...,[[0.60642857 0.62928571 0.62285714 ... 0.60214...
4,[0.30892857 0.29107143 0.29535714 ... 0.002857...,[[0.27464286 0.29285714 0.29428571 ... 0.28964...


In [ ]:
t_dropout_phi_real_gene

In [23]:
dfs = []
for data_name, f in zip(data_names, data_files):
    my_df = pd.read_csv(f, sep='\t')
    my_df.loc[:, 'model'] = data_name
#     my_df.columns = [data_name + '_' col_name for col_name in my_df.columns]
    dfs.append(my_df)
df = pd.concat(dfs, axis=0)
# df.info()

In [24]:
metrics = ['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']
h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
h1_hypothesis_bis = ['smaller', 'smaller', 'smaller', 'smaller', 'smaller']

df_nb = df.loc[df.model=='nb', metrics]
df_zinb = df.loc[df.model=='zinb', metrics]


from statsmodels.stats.weightstats import ttest_ind

def get_pvals(gby, other_df):
    my_df = gby[metrics]
#     print(my_df)
#     print(other_df)
    assert my_df.shape[1] == len(metrics)
    pvals = []
    for idx, alternative in enumerate(h1_hypothesis):
#         assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
        _, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
        pvals.append(pval)
    return np.array(pvals)

pvals_against_zinb = df.groupby('model').apply(get_pvals, other_df=df_zinb)
pvals_against_zinb = (pvals_against_zinb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

pvals_against_nb = df.groupby('model').apply(get_pvals, other_df=df_nb)
pvals_against_nb = (pvals_against_nb
         .apply(lambda x: pd.Series(x))
         .T)
pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

In [25]:
def get_summary(gby):
    res = {}
    res['mean'] = gby.mean()
    res['std'] = gby.std()
    res['pvals_against_nb'] = pvals_against_nb[gby.name]
    res['pvals_against_zi'] = pvals_against_zinb[gby.name]
    return pd.DataFrame(res).T

# df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
#                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
df_summary = df.groupby('model')[metrics].apply(get_summary)
df_summary


def my_styler(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
#     print(val)
    style = ['background-color: yellow' if v < 0.05 else '' for v in val]
    return style

df_summary = df_summary.stack().unstack(1).sort_index(level=1).swaplevel()
style = df_summary.style.apply(my_styler, subset=['pvals_against_nb', 'pvals_against_zi'], axis=1)
style
# df_summary.loc[pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :]].style.background_gradient()
# df_summary.style.apply(my_styler, subset=pd.IndexSlice[:, 'pvals'], axis=0)
# df_summary = df_summary.style.apply(my_styler, subset=pd.IndexSlice[pd.IndexSlice[:, 'pvals'], :], axis=1).render()
# df_summary.pivot(index=['model', ['mean', 'std', 'pvals']], columns=metrics)
# style

In [26]:
data_folder

NameError: name 'data_folder' is not defined

In [ ]:
# save as png
import imgkit
imgkit.from_string(style, os.path.join(data_path, '{}.png'.format(dataset_name)))

In [ ]:
# save as latex table

In [ ]:
text_file = open(os.path.join(data_path, 'latex_table.txt'), "w")
text_file.write(df_summary.to_latex())
text_file.close()